In [ ]:
import cv2
import datetime as dt
import json, os
import glob
import pandas as pd
from urllib.parse import urlparse
from PIL import Image, ImageDraw, ImageFont
from multiprocessing import Pool
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils

THUMBNAIL_WIDTH = 512
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000
X_PADDING_FULLRES = 190
Y_PADDING_FULLRES = 140
X_PADDING = X_PADDING_FULLRES * float(THUMBNAIL_WIDTH / PIXEL_COUNT_WIDTH)
Y_PADDING = Y_PADDING_FULLRES * float(THUMBNAIL_WIDTH / PIXEL_COUNT_HEIGHT)
ROOT_DIR = '/root/data/s3'
OUTPUT_BASE_DIR = 'imr_jan_27'
INBOUND_BUCKET = 'aquabyte-frames-resized-inbound'
VIDEO_BUCKET = 'aquabyte-images-adhoc'
UPLOAD_TO_S3 = False

s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))


def _get_bucket_key(url):
    parsed_url = urlparse(url, allow_fragments=False)
    if parsed_url.netloc.startswith('s3'):
        url_components = parsed_url.path.lstrip('/').split('/')
        bucket, key = url_components[0], os.path.join(*url_components[1:])
    else:
        bucket = parsed_url.netloc.split('.')[0]
        key = parsed_url.path.lstrip('/')
    return bucket, key


def process_s3_key_dir(s3_key_dir):
    try:
        left_f = s3_access_utils.download_from_s3(INBOUND_BUCKET, os.path.join(s3_key_dir,
            'left_frame.resize_512_512.jpg'))
        right_f = s3_access_utils.download_from_s3(INBOUND_BUCKET, os.path.join(s3_key_dir,
            'right_frame.resize_512_512.jpg'))
        crop_metadata_f = s3_access_utils.download_from_s3(INBOUND_BUCKET, os.path.join(s3_key_dir,
            'crops.json'))
    except Exception as e:
        print(e)
        return


    # open images and metadata files
    left_im = Image.open(left_f)
    right_im = Image.open(right_f)
    crop_metadata = json.load(open(crop_metadata_f))
    try:
        depth = crop_metadata['capture']['sensors'].get('aquabyte_depth_meters')
    except:
        depth = 'Depth not found'

    # draw boxes on images
    left_draw = ImageDraw.Draw(left_im)
    right_draw = ImageDraw.Draw(right_im)
    anns = crop_metadata['annotations']
    if anns:
        for ann in anns:
            c1 = max(ann['bbox'][0] - X_PADDING, 0)
            c2 = max(ann['bbox'][1] - Y_PADDING, 0)
            c3 = min(ann['bbox'][0] + ann['bbox'][2] + X_PADDING, THUMBNAIL_WIDTH)
            c4 = min(ann['bbox'][1] + ann['bbox'][3] + Y_PADDING, THUMBNAIL_WIDTH)
            if ann['image_id'] == 1:
                left_draw.rectangle([(c1, c2), (c3, c4)])
            elif ann['image_id'] == 2:
                right_draw.rectangle([(c1, c2), (c3, c4)])

    # stitch images
    result = Image.new('RGB', (2*THUMBNAIL_WIDTH, THUMBNAIL_WIDTH))
    result.paste(im=left_im, box=(0, 0))
    result.paste(im=right_im, box=(THUMBNAIL_WIDTH, 0))

    # write timestamp on stitched image
    result_draw = ImageDraw.Draw(result)
    ts = [c for c in left_f.split('/') if c.startswith('at=')][0]
    display_ts = 'UTC Time: {}'.format(ts.replace('at=', ''))
    display_depth = 'Depth: {}m'.format(depth)
    result_draw.text((0, 0), display_ts, (255, 255, 255))
    result_draw.text((0, 10), display_depth, (255, 255, 255))

    output_f = left_f.replace(ROOT_DIR, OUTPUT_BASE_DIR).replace('left_', 'stereo_')
    if not os.path.exists(os.path.dirname(output_f)):
        os.makedirs(os.path.dirname(output_f))
    result.save(output_f)

def generate_video(pen_id, date):
    query = """
        SELECT captured_at, left_crop_url
        FROM prod.crop_annotation ca
        WHERE ca.pen_id={} AND ca.service_id = 2 AND ca.captured_at > '{}'
        LIMIT 1;
    """.format(pen_id, date)
    print(query)
    print('Extracting s3 keys...')
    df = rds_access_utils.extract_from_database(query)
    print(df)
    image_url = df.left_crop_url.iloc[0]
    print(image_url)
    bucket, key = _get_bucket_key(image_url)

    s3_folder = os.path.join(key[:key.index('date')], 'date={}'.format(date))
    print(s3_folder)
    generator = s3_access_utils.get_matching_s3_keys(INBOUND_BUCKET, s3_folder, suffixes=['capture.json'])
    keys = [key for key in generator]
    s3_key_dirs = sorted(list(set([os.path.dirname(f) for f in keys])))
    print('S3 keys extraction complete!')

    print('Generating frames...')
    # generate frames
    pool = Pool(20)
    pool.map(process_s3_key_dir, s3_key_dirs)
    print('Frame generation complete!')

    print('Generating video...')
    # generate video
    image_fs = sorted(filter(lambda path: 'stereo' in path, glob.glob(os.path.join(OUTPUT_BASE_DIR, '**', '*.jpg'), recursive=True)))
    print('\n\nIMAGE FILES')
    print(image_fs)
    im = cv2.imread(image_fs[0])
    height, width, layers = im.shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    video_f = os.path.join(OUTPUT_BASE_DIR, 'pen_id_{}_date_{}_videofull.avi'.format(str(pen_id), date))
    video = cv2.VideoWriter(video_f, fourcc, 4, (width, height), True)
    for idx, image_f in enumerate(image_fs):
        if idx % 1000 == 0:
            print(idx)
        im = cv2.imread(image_f, cv2.IMREAD_COLOR)
        video.write(im)

    cv2.destroyAllWindows()
    video.release()
    print('Video generation complete!')

    if UPLOAD_TO_S3:
        # upload to s3
        video_key = os.path.join('videos', str(pen_id), os.path.basename(video_f))
        s3_access_utils.s3_client.upload_file(video_f, VIDEO_BUCKET, video_key)


# def main(pen_ids, days_back=60):
#     date = dt.datetime.now() - dt.timedelta(days_back)
#     date = date.strftime('%Y-%m-%d')
def main(pen_ids, date):
    print(f'Building video for days since {date} for {pen_ids}')

    for pen_id in pen_ids:
        print(f'Generating pen:{pen_id} video starting at {date}')
        generate_video(pen_id, date)


    

In [ ]:
main([61], '2020-01-27')

In [ ]:
s3_access_utils.s3_client.upload_file(video_f, VIDEO_BUCKET, video_key)

In [ ]:
video_f = '/root/alok/repos/cv_research/alok/biomass_estimation/playground/imr_jan_27/pen_id_61_date_2020-01-27_videofull.avi'
s3_access_utils.s3_client.upload_file(video_f, VIDEO_BUCKET, 'videos/61/pen_id_61_date_2020-01-27_videofull.avi')
